This is Ipython Notebook five of five for my final project in General Assembly's data science course.

The following code contains contains the data modeling.

In [1]:
# imports
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV 
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# allow plots to appear directly in the notebook
import seaborn as sns
%matplotlib inline

In [6]:
data = pd.read_csv(r'C:\Users\alsherman\Desktop\GitHub\DataScience_GeneralAssembly\Data\Cleaned_Data_Final.csv')

In [7]:
data.describe()

,ID,country,latitude,location_data_accuracy,longitude,availability,average_image_size,bathroom,bedroom,cat,...,price,smoking,square_footage,grocery_list,gym_list,square_footage_cleaned,weekday_of_posting,Posting_Time_AM_or_PM,Posting_Time,Posting_Day
count,4.357000e+03,4357.000000,4357.000000,4357.000000,4357.000000,4357,4357.000000,4357.000000,4357.000000,4357.000000,...,4357.000000,4357.000000,4357.000000,4357.000000,4357.000000,4357.000000,4357.000000,4357.000000,4357.000000,4357.000000
mean,5.001185e+09,0.642415,31.059039,3.904521,-61.518204,1,235049.345880,1.362405,1.575166,0.627496,...,1684.567592,0.714253,757.251549,10.279780,6.939867,906.553133,4.367914,0.673858,6.742369,8.739729
std,1.038692e+07,0.479344,15.717873,7.123956,31.154376,0,90647.748582,0.695859,1.023219,0.483527,...,364.640486,0.451821,528.436344,8.028187,7.248670,397.481536,1.637151,0.468854,3.756868,10.203315
min,4.959352e+09,0.000000,-85.050342,0.000000,-96.045800,1,0.000000,0.000000,0.000000,0.000000,...,1005.000000,0.000000,0.000000,0.000000,0.000000,58.000000,0.000000,0.000000,1.000000,1.000000
25%,5.000034e+09,0.000000,38.779400,0.000000,-77.121000,1,270000.000000,1.000000,1.000000,0.000000,...,1399.000000,0.000000,503.000000,2.000000,0.000000,660.000000,4.000000,0.000000,3.250000,2.000000
50%,5.006199e+09,1.000000,38.895463,0.000000,-77.043624,1,270000.000000,1.000000,1.000000,1.000000,...,1650.000000,1.000000,769.000000,11.000000,4.000000,840.000000,5.000000,1.000000,6.400000,3.000000
75%,5.006999e+09,1.000000,38.973231,5.000000,-76.884000,1,270000.000000,2.000000,2.000000,1.000000,...,1949.000000,1.000000,1012.000000,20.000000,12.000000,1061.000000,6.000000,1.000000,10.290000,17.000000
max,5.008221e+09,1.000000,41.333300,22.000000,179.992769,1,270000.000000,4.500000,6.000000,1.000000,...,2499.000000,1.000000,9000.000000,20.000000,20.000000,9000.000000,6.000000,1.000000,12.590000,30.000000


In [8]:
# Create dummy variables

data = pd.concat([data, pd.get_dummies(data.housing_type)], axis=1)
data = pd.concat([data, pd.get_dummies(data.laundry)], axis=1)
data = pd.concat([data, pd.get_dummies(data.parking)], axis=1)

data[0:1]

,ID,city,country,latitude,location_data_accuracy,longitude,state,availability,average_image_size,bathroom,...,laundry on site,w/d hookups,w/d in unit,No_Parking_Data,attached garage,carport,detached garage,off-street parking,street parking,valet parking
0,4959351766,No_City_Data,0,0,0,0,No_State_Data,1,270000,1,...,0,0,0,1,0,0,0,0,0,0


In [9]:
feature_cols = [
'latitude',
'longitude',
'bathroom',
'bedroom',
'image_number',
#'square_footage', 
'square_footage_cleaned',
'grocery_list',
'gym_list',  
'attached garage',
'off-street parking',
'street parking',     
'Posting_Time_AM_or_PM',
'Posting_Time',
'Posting_Day',
'weekday_of_posting'
]

## Train Test Split

In [10]:
# create X and y
X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

##Random Forest

In [11]:
# import class, instantiate estimator, fit with all data
rfclf = RandomForestClassifier(n_estimators=200, max_features=5, oob_score=True, random_state=1)
rfclf.fit(X_train, y_train)

# compute the out-of-bag classification accuracy
print rfclf.oob_score_
print len(feature_cols)
print len(rfclf.feature_importances_)

0.171717171717
15
15


In [12]:
# compute the feature importances
Feature_Importance = pd.DataFrame({'feature':feature_cols, 'importance':rfclf.feature_importances_})
Feature_Importance.sort('importance', ascending=False)

,feature,importance
5,square_footage_cleaned,0.149799
12,Posting_Time,0.145246
4,image_number,0.106396
0,latitude,0.104034
1,longitude,0.103910
13,Posting_Day,0.071370
7,gym_list,0.062087
6,grocery_list,0.059932
14,weekday_of_posting,0.056020
3,bedroom,0.036576


## Linear Regression

In [66]:
### SCIKIT-LEARN ###

# instantiate and fit
lm2 = LinearRegression()
lm2.fit(X_train, y_train)

y_pred = lm2.predict(X_test) # Predict
# print the intercept
#print lm2.intercept_

# pair the feature names with the coefficients
#print zip(feature_cols, lm2.coef_)

# Access accuracy
y_pred = [int(y) for y in y_pred]
y_test = [int(y) for y in y_test]

print metrics.accuracy_score(y_test, y_pred)

0.00183486238532


##Ridge Regression

In [38]:
rreg = Ridge(alpha=0.01, normalize=True) 
rreg.fit(X_train, y_train) 
rreg.coef_ 
preds = rreg.predict(X_test) 
print 'RMSE (Ridge reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)) 

# Access accuracy
preds = [int(y) for y in preds]
y_test = [int(y) for y in y_test]
print metrics.accuracy_score(y_test, preds)

RMSE (Ridge reg.) = 290.568752123
0.0


In [39]:
alpha_range = 10.**np.arange(-2, 3) 
rregcv = RidgeCV(normalize=True, scoring='mean_squared_error', alphas=alpha_range) 
rregcv.fit(X_train, y_train) 
print rregcv.alpha_ 
preds = rregcv.predict(X_test) 
print 'RMSE (Ridge CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)) 

0.1
RMSE (Ridge CV reg.) = 292.339077627


##Lasso Regression

In [40]:
from sklearn.linear_model import Lasso 
las = Lasso(alpha=0.01, normalize=True) 
las.fit(X_train, y_train) 
las.coef_ 
preds = las.predict(X_test) 
print 'RMSE (Lasso reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds))  
  
from sklearn.linear_model import LassoCV 
lascv = LassoCV(normalize=True, alphas=alpha_range) 
lascv.fit(X_train, y_train) 
print lascv.alpha_ 
print zip(feature_cols,lascv.coef_) 
preds = lascv.predict(X_test) 
print 'RMSE (Lasso CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)) 

# Access accuracy
preds = [int(y) for y in preds]
y_test = [int(y) for y in y_test]
print metrics.accuracy_score(y_test, preds)

RMSE (Lasso reg.) = 290.467137839
0.1
[('latitude', -35.412229861527386), ('longitude', 50.790822299227464), ('bathroom', 74.458408353096075), ('bedroom', 107.9424230345919), ('image_number', 7.3968920475598088), ('square_footage_cleaned', 0.15351606250238639), ('grocery_list', -0.0), ('gym_list', 27.131736553781522), ('attached garage', 159.07060582591009), ('off-street parking', -49.833490438071173), ('street parking', -14.661270717662225), ('Posting_Time_AM_or_PM', 0.0), ('Posting_Time', -0.0), ('Posting_Day', -0.30952752911817527), ('weekday_of_posting', 0.0)]
RMSE (Lasso CV reg.) = 290.819138529
0.00508474576271


## Depreciated Models

##Naive Bayes

In [14]:
 #saving for future use case

#Create Train Test split for text data (listing description)

naive_bayes_feature_cols = 'description'

# create X and y
X = data[naive_bayes_feature_cols]
#y = data

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

TypeError: Expected sequence or array-like, got 1856

In [47]:
# COUNTVECTORIZER: 'convert text into a matrix of token counts'
# instantiate the vectorizer
vect = CountVectorizer()

# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train)
train_dtm

# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test)
test_dtm

# store feature names and examine them
train_features = vect.get_feature_names()
len(train_features)
train_features[:50]
train_features[-50:]

# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

# examine the vocabulary and document-term matrix together
pd.DataFrame(train_dtm.toarray(), columns=vect.get_feature_names()).head()

,00,000,0009,000pets,001114,001255887315,001368910757,002,004,005,...,zyq9z,zz,¼gather,área,â¼,â½,ãƒâ,über,ýntelligence,šã
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
nb = MultinomialNB()
nb.fit(train_dtm, y_train)

# make predictions on test data using test_dtm
y_pred = nb.predict(test_dtm)
y_pred

# compare predictions to true labels
print metrics.accuracy_score(y_test, y_pred)
#print metrics.confusion_matrix(y_test, y_pred)

0.0674715909091
